# Download and convert to Zarr
This downloads SWOT Pixel Cloud products from hydroweb.next (API-Key necessary) based on a region and a period of interest.
Then is extracts information contained in the area of interest for your study, stores everything in a Zarr Database (based on the zcollection package) for future use.
Zarr (and the way we partitionned data with zcollection) is very efficient for computation. However, it is not (yet) compatible with QGIS compared to Geopackage.


## Setting the region and period of interest
Using a geopackage layer, preliminary created with, e.g. QGIS, to limit data download and database

In [1]:
import pixcdust
from pixcdust.downloaders.hydroweb_next import PixCDownloader
import geopandas as gpd
from datetime import datetime

In [2]:
# reading the area of interest polygon
gdf_geom = gpd.read_file("../data/aoi.gpkg")

dates = (
    datetime(2023,4,6),
    datetime(2023,4,8),
)

## Download
This will unfortunately lead to downloading many big files (that will be removed later). This is the only way right now, but the hydroweb.next team is working on improving that.

In [3]:
pixcdownloader = PixCDownloader(
    gdf_geom,
    dates,
    verbose=1,
    path_download='/tmp/pixc',
    )
pixcdownloader.search_download()

Downloaded products:   0%|                                                                                    …

0.00B [00:00, ?B/s]

0.00B [00:00, ?B/s]

## Extraction
Now we have all necessary files, let us extract key variables within area of interest in a Zarr (zcollection) database.
This Zarr partionned format is very efficient for time analysis, but is not currently accessible in GIS softwares such as QGIS
We are using the same geodataframe to limit the data to the area of interest

In [4]:
from pixcdust.converters.zarr import Nc2ZarrConverter
from glob import glob

In [5]:
pixc = Nc2ZarrConverter(
            path_in = glob(pixcdownloader.path_download+'/*/*nc'),
            variables=['height', 'sig0', 'classification'],
            area_of_interest=gdf_geom,
        )
pixc.database_from_nc(path_out='/tmp/pixc_zarr')

/home/vschaffn/Documents/swot_pixc_study/pixc-env/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34147 instead
  warnings.warn(
Exception ignored in: <function CachingFileManager.__del__ at 0x7a90c1840b80>
Traceback (most recent call last):
  File "/home/vschaffn/Documents/swot_pixc_study/pixc-env/lib/python3.10/site-packages/xarray/backends/file_manager.py", line 250, in __del__
    self.close(needs_lock=False)
  File "/home/vschaffn/Documents/swot_pixc_study/pixc-env/lib/python3.10/site-packages/xarray/backends/file_manager.py", line 234, in close
    file.close()
  File "src/netCDF4/_netCDF4.pyx", line 2669, in netCDF4._netCDF4.Dataset.close
  File "src/netCDF4/_netCDF4.pyx", line 2636, in netCDF4._netCDF4.Dataset._close
  File "src/netCDF4/_netCDF4.pyx", line 2164, in netCDF4._netCDF4._ensure_nc_success
RuntimeError: NetCDF: HDF error
/home/vschaffn/Documents

database has been succesfully created, we can remove the raw files

In [6]:
# import shutil
# shutil.rmtree('/tmp/pixc')

# Read the database
previous steps are not necessary

Now we can open this database in a xarray, or dataframe, or GeoDataFrame

In [7]:
from pixcdust.readers.zarr import ZarrReader
import datetime

pixc_read = ZarrReader(
    "/tmp/pixc_zarr"
)
pixc_read.read((datetime.datetime(2023,4,6), datetime.datetime(2023,4,8)))
pixc_read.data

/home/vschaffn/Documents/swot_pixc_study/pixc-env/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33495 instead
  warnings.warn(


<xarray.Dataset> Size: 52MB
Dimensions:         (points: 1519115)
Dimensions without coordinates: points
Data variables:
    tile_number     (points) uint16 3MB dask.array<chunksize=(755197,), meta=np.ndarray>
    sig0            (points) float32 6MB dask.array<chunksize=(755197,), meta=np.ndarray>
    pass_number     (points) uint16 3MB dask.array<chunksize=(755197,), meta=np.ndarray>
    longitude       (points) float32 6MB dask.array<chunksize=(755197,), meta=np.ndarray>
    height          (points) float32 6MB dask.array<chunksize=(755197,), meta=np.ndarray>
    time            (points) datetime64[ns] 12MB dask.array<chunksize=(755197,), meta=np.ndarray>
    cycle_number    (points) uint16 3MB dask.array<chunksize=(755197,), meta=np.ndarray>
    classification  (points) float32 6MB dask.array<chunksize=(755197,), meta=np.ndarray>
    latitude        (points) float32 6MB dask.array<chunksize=(755197,), meta=np.ndarray>
Attributes:
    azimuth_offset:              3
    description:                 cloud of geolocated interferogram pixels
    interferogram_size_azimuth:  3245
    interferogram_size_range:    4857
    looks_to_efflooks:           1.5340684990936673
    num_azimuth_looks:           7.0

In [8]:
gdf_pixc = pixc_read.to_geodataframe()
gdf_pixc

/home/vschaffn/Documents/swot_pixc_study/pixcdust/readers/base_reader.py:115: UserWarning: No active geometry column to be set. The resulting object will be a pandas.DataFrame with geopandas.GeometryArray(s) containing geometry and CRS information. Use `.set_geometry()` to set an active geometry and upcast to the geopandas.GeoDataFrame manually.
  gdf = self.data.xvec.to_geodataframe()


,tile_number,sig0,pass_number,longitude,height,time,cycle_number,classification,latitude
points,,,,,,,,,
0,78,4.979378,16,1.504697,205.690842,2023-04-06 09:46:18,482,1.0,43.517670
1,78,1.716195,16,1.503559,201.013336,2023-04-06 09:46:18,482,1.0,43.517670
2,78,1.432429,16,1.503695,208.560715,2023-04-06 09:46:18,482,1.0,43.517693
3,78,7.839147,16,1.503859,205.756393,2023-04-06 09:46:18,482,3.0,43.517723
4,78,4.303982,16,1.504143,206.650543,2023-04-06 09:46:18,482,3.0,43.517773
...,...,...,...,...,...,...,...,...,...
1519110,78,0.414571,16,1.380191,168.646057,2023-04-07 09:36:56,483,1.0,43.686825
1519111,78,0.441882,16,1.380251,169.712540,2023-04-07 09:36:56,483,1.0,43.686836
1519112,78,0.232018,16,1.380750,171.547318,2023-04-07 09:36:56,483,1.0,43.686520


Enjoy!